In [89]:
import os
import sys
from tqdm import tqdm
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import clear_output

sys.path.append('src')
from train_test import train_test_split

from ncf_helpers import create_train_step, create_test_step, create_inference, get_ratings
from metrics import precision_recall_at_k, hit_rate_total, arhr_total, ndcg_at_k
# from metrics import

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
DATA_PATH = './data.nosync/'

In [3]:
train = pd.read_csv(DATA_PATH + 'train.csv')
valid = pd.read_csv(DATA_PATH + 'valid.csv')
test = pd.read_csv(DATA_PATH + 'test.csv')
lastfm_360_behav = pd.read_csv(DATA_PATH + 'behav-360k-processed.csv')
lastfm_360_demo = pd.read_csv(DATA_PATH + 'demo-360k-processed.csv')
lastfm_360_demo = lastfm_360_demo.set_index('user_email').astype(bool).reset_index()
test_users = np.load(DATA_PATH + 'test_users.npy')

In [4]:
train_plays_max = train['log_plays'].max()
train['log_plays'] = train['log_plays'] / train_plays_max
valid['log_plays'] = valid['log_plays'] / train_plays_max
test['log_plays'] = test['log_plays'] / train_plays_max

In [5]:
artists = lastfm_360_behav['artist_id'].unique()
users = lastfm_360_demo['user_email'].unique()

In [108]:
k = 10

In [109]:
def baseline_predict(data, k=100):
    total_log_plays = data.groupby('artist_id').sum()
    ranked_artist_scores = total_log_plays.sort_values(['log_plays'], ascending=False)['log_plays']
    return ranked_artist_scores[:k]

In [106]:
pred = baseline_predict(test)

In [107]:
pred_ratings = {u: np.array([pred.index.values, pred.values]) for u in test_users}

In [102]:
compute_metrics(test, test_users, pred_ratings, k)

Computing precision & recall...



Computing normalized discounted cumulative gain...



Computing hit rate...



Computing average reciprocal hit ranking...



Metrics: 
Precision @ 20: 0.01345
Recall    @ 20: 0.04913571428571429
Ndcg @ 20: 0.09103585540079194
Hit rate: 0.269
Arhr: 0.060413446980436256
